In [296]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import pydae.svg_tools as svgt
#%config InlineBackend.figure_format = 'svg'
import pydae.grid_tools as gt
import scipy.optimize as sopt
from scipy.optimize import NonlinearConstraint
import time
import json
plt.style.use('ieee.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [297]:
%matplotlib widget

In [298]:
from pydae import ssa
from onehalf import onehalf_class

In [299]:
grid = onehalf_class()

#H = T_p/K_p/2
H_03 = 0.1; # desired virtual inertia 
K_p_03 = 0.8; # active power proportinal gain
T_p_03 = K_p_03*2*H_03;  # active power integral time constant

params = {
          "S_n_01":10e6,"K_p_agc":0.01,"K_i_agc":0.01,
          "K_delta_01":0.01,"p_c_01":0.01,'K_sec_01':0.1,"K_imw_01":0.0,"K_a_01":200,
          'D_01':0,
           "P_02":-8.0e6,"Q_02":0.0,
        # UVSG 03:
          "S_n_03":2e6,"R_s_03":0.01, # VSC
          "R_v_03":0.0,"X_v_03":0.1,"K_p_03":K_p_03,"T_p_03":T_p_03, # PI-VSG
          "R_uc_03":0.001,'K_u_0_03':0.005,'K_u_max_03':10,"C_u_03":514, # UCAP ans SoC control
          'p_gin_0_03': 1.0,'p_g_ref_03': 0.001,"T_cur_03":1000,  # primary source
          'q_s_ref_03': 0.0, # reactive power
          'R_lim_03':0.02,"K_h_03":10,  # HFPS
          "Droop_03":1000.04,'DB_03':0.015, "K_fpfr_03":1.0, # PFR
          "v_u_ref_03":126.491,
          "V_u_lt_03":81,"V_u_ht_03":159
          }


gt.change_line(grid,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
grid.initialize([params],'xy_0.json',compile=True)
S_b_01_mva = grid.get_value('S_n_01')/1e6
S_b_03_mva = grid.get_value('S_n_03')/1e6
S_b_sys_mva = grid.get_value('S_base')/1e6
#grid.save_0('xy_0.json')
#grid.report_y()
#ssa.eval_A(grid)
#ssa.damp_report(grid)
#ssa.participation(grid).abs().round(2)
grid.report_params()

S_base = 100000.00
g_01_02 = 124.92
b_01_02 = -4996.88
bs_01_02 =  0.00
g_02_03 = 124.92
b_02_03 = -4996.88
bs_02_03 =  0.00
U_01_n = 400.00
U_02_n = 400.00
U_03_n = 400.00
S_n_01 = 10000000.00
Omega_b_01 = 314.16
H_01  =  6.50
T1d0_01 =  8.00
T1q0_01 =  0.40
X_d_01 =  1.80
X_q_01 =  1.70
X1d_01 =  0.30
X1q_01 =  0.55
D_01  =  0.00
R_a_01 =  0.00
K_delta_01 =  0.01
K_sec_01 =  0.10
K_a_01 = 200.00
K_ai_01 =  0.00
T_r_01 =  0.02
V_min_01 = -10000.00
V_max_01 =  5.00
K_aw_01 = 10.00
Droop_01 =  0.05
T_gov_1_01 =  1.00
T_gov_2_01 =  2.00
T_gov_3_01 = 10.00
K_imw_01 =  0.00
omega_ref_01 =  1.00
T_wo_01 = 10.00
T_1_01 =  0.10
T_2_01 =  0.10
K_stab_01 =  1.00
S_n_03 = 2000000.00
Omega_b_03 = 314.16
K_p_03 =  0.80
T_p_03 =  0.16
K_q_03 =  0.10
T_q_03 =  0.10
X_v_03 =  0.10
R_v_03 =  0.00
R_s_03 =  0.01
C_u_03 = 514.00
K_u_0_03 =  0.01
K_u_max_03 = 10.00
V_u_min_03 = 80.00
V_u_max_03 = 160.00
R_uc_03 =  0.00
K_h_03 = 10.00
R_lim_03 =  0.02
V_u_lt_03 = 81.00
V_u_ht_03 = 159.00
Droop_03 = 1000.0

## PFR Design

### Simulation without PFR

In [300]:
## Without PFR by CIG
Dp_pert = -0.2 # p.u. installed power capacity base 
t_pert = 1.0

grid = onehalf_class()
gt.change_line(grid,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid.initialize([params],'xy_0.json',compile=True)
P_load_0_mva = grid.get_value('P_02')/1e6
grid.run([{'t_end':t_pert,'Dt':0.01,'decimation':1}])
grid.run([{'t_end':10.0,'P_01':P_load_0_mva*(0-Dp_pert)*1e6}])
grid.run([{'t_end':60.1,'Dt':0.1}])
grid.post();

In [310]:
(t_nadir_ref_end-t_nadir_ref)*p_pfr_mw/2

2.7375408804161725

### PFR design

In [301]:
## Design of PFR by CIG
omega_nadir_ref = 0.982
Droop = 0.003 # in CIG base
mid_pos = 0.5
K_ol = 1.2

S_n_syn_mva = grid.get_value('S_n_01')/1e6
S_n_cig_mva = grid.get_value('S_n_03')/1e6
S_n_sys_mva = S_n_syn_mva + S_n_cig_mva

# importan points
idx_pert = np.searchsorted(grid.T[:,0],t_pert)
idx_nadir = np.argmin(grid.get_values('omega_coi'))
t_nadir = grid.T[idx_nadir]
p_m_pert = grid.get_values('p_m_01')[idx_pert]*S_b_01_mva
omega_nadir = grid.get_values('omega_coi')[idx_nadir]

t_nadir_ref = grid.T[idx_pert:idx_nadir,0][np.argmin(grid.get_values('omega_coi')[idx_pert:idx_nadir]>omega_nadir_ref)]
idx_nadir_ref = np.searchsorted(grid.T[:,0],t_nadir_ref)

t_nadir_ref_end = grid.T[idx_nadir:,0][np.argmin(grid.get_values('omega_coi')[idx_nadir:]<omega_nadir_ref)]
idx_nadir_ref_end = np.searchsorted(grid.T[:,0],t_nadir_ref_end)

# Load
P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
p_pfr_mw = P_load[idx_nadir_ref] - grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva
p_pfr_pu = p_pfr_mw/S_n_cig_mva
omega_ref = omega_nadir_ref

# FPFR parameters
P_f_max = p_pfr_pu*K_ol  # pu in CIG power base
omega_ll = omega_ref + Droop*p_pfr_pu
omega_lh = omega_ref - Droop*(P_f_max - p_pfr_pu)
DB = (1 - omega_ll)*2

print(f'omega_ll = {omega_ll:0.4f}, omega_ref = {omega_ref:0.4f}, omega_lh = {omega_lh:0.4f}')
print(f'p_pfr_pu = {p_pfr_pu:0.4f},P_f_max = {P_f_max:0.4f}, DB = {DB:0.4f}')

# Obtained PFR curve
omega_hh = 1 + DB/2
omega_h_ref = 1 + (1-omega_ref)
omega_hl = 1 + (1-omega_lh)
omegas = np.array([      0,   omega_lh, omega_ref,omega_ll,1,omega_hh,omega_h_ref,omega_hl,2])
p_fs   = np.array([P_f_max,    P_f_max,  p_pfr_pu,       0,0,       0,  -p_pfr_pu, -P_f_max,    -P_f_max])

omega_ll = 0.9837, omega_ref = 0.9820, omega_lh = 0.9817
p_pfr_pu = 0.5542,P_f_max = 0.6650, DB = 0.0327


In [308]:
fig,axes = plt.subplots(nrows=2)
axes[0].plot(grid.T,grid.get_values('omega_coi'),label='$f_{coi}$')


axes[0].plot(grid.T[idx_pert],grid.get_values('omega_coi')[idx_pert],'o',color=colors[5] )
axes[0].plot(grid.T[idx_nadir],grid.get_values('omega_coi')[idx_nadir],'o',color=colors[3])
axes[0].plot(grid.T[idx_nadir_ref],grid.get_values('omega_coi')[idx_nadir_ref],'o',color=colors[2] )

P_load_mw = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
P_m_mw = grid.get_values('p_m_01')*S_b_01_mva

axes[1].plot(grid.T,P_load_mw,label='$p_l$')

axes[1].plot(grid.T,P_m_mw,label='$p_m$')
#axes[1].plot(grid_pfr.T,grid_pfr.get_values('p_m_01')*S_b_01_mva,label='p_m_01')

axes[1].plot(grid.T[idx_nadir_ref],P_m_mw[idx_nadir_ref],'o',color=colors[2])
axes[1].plot(grid.T[idx_nadir_ref],P_load_mw[idx_nadir_ref],'o',color=colors[2])

axes[1].plot(grid.T[idx_nadir],P_load_mw[idx_nadir],'o',color=colors[3])

axes[1].plot(grid.T[idx_nadir_ref_end],P_load_mw[idx_nadir_ref_end],'o',color='#888')

#axes[2].plot(grid_pfr.T,grid_pfr.get_values('p_s_03')*S_b_03_mva)
#axes[1].annotate(s='', xy=(1,1), xytext=(0,0), arrowprops=dict(arrowstyle='<->'))

axes[1].annotate("",
            xy=(grid.T[idx_nadir_ref], P_m_mw[idx_nadir_ref]), xycoords='data',
            xytext=(grid.T[idx_nadir_ref], P_load_mw[idx_nadir_ref]), textcoords='data',
            arrowprops=dict(arrowstyle='<|-|>, head_width=0.1', facecolor='black')
            )

for ax in axes:
    ax.grid()
    ax.set_xlim(0,30)
#axes[1].plot(grid.T,grid.get_values('p_s_03')*S_b_03_mva,label='p_s_03')

axes[1].fill_between(grid.T[idx_nadir_ref:idx_nadir,0], 
                     P_m_mw[idx_nadir_ref:idx_nadir], 
                     P_load_mw[idx_nadir_ref:idx_nadir], 
                     where=(P_m_mw[idx_nadir_ref:idx_nadir] < P_load_mw[idx_nadir_ref:idx_nadir]),
                color=colors[2], alpha=0.5)

axes[1].fill_between(grid.T[idx_nadir:idx_nadir_ref_end,0], 
                     P_m_mw[idx_nadir:idx_nadir_ref_end], 
                     P_load_mw[idx_nadir:idx_nadir_ref_end], 
                     where=(P_m_mw[idx_nadir:idx_nadir_ref_end] > P_load_mw[idx_nadir:idx_nadir_ref_end]),
                color='#AAA', alpha=0.5)

axes[0].legend()
axes[1].legend()
axes[0].set_ylabel('Frequency (pu)')
axes[1].set_ylabel('Powers (MW)')

axes[1].set_xlabel('Time (s)')

fig.align_ylabels()
fig.tight_layout()
fig.savefig('../'*5 + 'docs/fpfr/figs/without_pfr.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Storage design

In [303]:
Dtime_pfr_estimated = grid.T[idx_nadir_ref:idx_nadir,0]-grid.T[idx_nadir_ref,0]
P_pfr_estimated = P_load_mw[idx_nadir_ref:idx_nadir] - P_m_mw[idx_nadir_ref:idx_nadir]  
e_pfr_estimated_1 = np.trapz(P_pfr_estimated,x=Dtime_pfr_estimated)

Dtime_pfr_estimated = grid.T[idx_nadir:idx_nadir_ref_end,0]-grid.T[idx_nadir,0]
P_pfr_estimated = P_load_mw[idx_nadir:idx_nadir_ref_end] - P_m_mw[idx_nadir:idx_nadir_ref_end]  
e_pfr_estimated_2 = np.trapz(P_pfr_estimated,x=Dtime_pfr_estimated)

e_pfr_estimated = e_pfr_estimated_1 - e_pfr_estimated_2

print(f'e_pfr_estimated = {e_pfr_estimated:0.3f} MJ')

# E_u_discharge = 0.5*C_u*(V_u_0**2 - V_u_min**2)
E_u_discharge =  e_pfr_estimated*1e6
V_u_min = 80
V_u_max = 160
#SoC = (v_u**2 - V_u_min**2)/(V_u_max**2 - V_u_min**2) 
SoC_0 = 0.5

V_u_0 = np.sqrt(SoC_0*(V_u_max**2 - V_u_min**2)+V_u_min**2)
print(f'V_u_0 = {V_u_0:0.3f} V')

C_u = E_u_discharge/(0.5*(V_u_0**2 - V_u_min**2))
print(f'C_u = {C_u:0.3f} F')

e_pfr_estimated = 2.468 MJ
V_u_0 = 126.491 V
C_u = 514.199 F


## Results with PFR

In [304]:
params_pfr = params.copy()
params_pfr.update({'DB_03':DB,'Droop_03':Droop,'P_f_max_03':P_f_max})
grid_pfr = onehalf_class()
gt.change_line(grid_pfr,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid_pfr,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid_pfr.initialize([params_pfr],'xy_0.json',compile=True)
grid_pfr.run([{'t_end':1.0,'Dt':0.01,'decimation':1}])
grid_pfr.run([{'t_end':10.0,'P_01':P_load_0_mva*(0-Dp_pert)*1e6}])
grid_pfr.run([{'t_end':60.1,'Dt':0.1}])
grid_pfr.post();





### Frequency and powers

In [305]:
plt.close('all')
fig,axes = plt.subplots(nrows=4, figsize=(6.4, 4.8*1.75))
t_ini = 0
t_end = 45

# Frequency
axes[0].plot(grid.T,grid.get_values('omega_coi'),label='w/o PFR',color=colors[2])
axes[0].plot(grid_pfr.T,grid_pfr.get_values('omega_coi'),label='with PFR',color=colors[1])

axes[0].plot(grid.T[idx_pert],grid.get_values('omega_coi')[idx_pert],'o',color=colors[5] )
axes[0].plot(grid.T[idx_nadir],grid.get_values('omega_coi')[idx_nadir],'o',color=colors[3])
axes[0].plot(grid.T[idx_nadir_ref],grid.get_values('omega_coi')[idx_nadir_ref],'o',color=colors[2] )
axes[0].plot(grid.T[idx_nadir_ref_end],grid.get_values('omega_coi')[idx_nadir_ref_end],'o',color='#888')

# Powers
P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
axes[1].plot(grid.T,P_load,label='$p_l$',color=colors[0])

axes[1].plot(grid.T,grid.get_values('p_m_01')*S_b_01_mva,label='$p_m$ w/o PFR',color=colors[2])
axes[1].plot(grid_pfr.T,grid_pfr.get_values('p_m_01')*S_b_01_mva,label='$p_m$ with PFR',color=colors[1])

axes[1].plot(grid.T[idx_pert],grid.get_values('p_m_01')[idx_pert]*S_b_01_mva,'o',color=colors[5] )
axes[1].plot(grid.T[idx_nadir],grid.get_values('p_m_01')[idx_nadir]*S_b_01_mva,'o',color=colors[3])
axes[1].plot(grid.T[idx_nadir_ref],grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva,'o',color=colors[2] )
axes[1].plot(grid.T[idx_nadir_ref_end],grid.get_values('p_m_01')[idx_nadir_ref_end]*S_b_01_mva,'o',color='#888')
axes[1].plot(grid_pfr.T[idx_nadir_ref_end],grid_pfr.get_values('p_m_01')[idx_nadir_ref_end]*S_b_01_mva,'o',color='#888')

# CIG Power
axes[2].plot(grid_pfr.T,grid_pfr.get_values('p_s_03')*S_b_03_mva,color=colors[1])
axes[2].plot(grid.T[idx_pert],grid_pfr.get_values('p_s_03')[idx_pert]*S_b_03_mva,'o',color=colors[5] )
axes[2].plot(grid.T[idx_nadir],grid_pfr.get_values('p_s_03')[idx_nadir]*S_b_03_mva,'o',color=colors[3])
axes[2].plot(grid.T[idx_nadir_ref],grid_pfr.get_values('p_s_03')[idx_nadir_ref]*S_b_03_mva,'o',color=colors[2] )
axes[2].plot(grid.T[idx_nadir_ref_end],grid_pfr.get_values('p_s_03')[idx_nadir_ref_end]*S_b_03_mva,'o',color='#888')
axes[2].plot([t_ini,t_end],[p_pfr_pu*S_b_03_mva]*2,color='#666')
axes[2].plot([t_ini,t_end],[P_f_max*S_b_03_mva]*2,color=colors[0])

axes[2].annotate("$p_f^{\star}$",
            xy=(23,p_pfr_pu*S_b_03_mva), xycoords='data',
            xytext=(23,0.7), textcoords='data',ha="center",color='#444',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#666', edgecolor='#666')
            )

axes[2].annotate("$P_f^{\max}$",
            xy=(33,P_f_max*S_b_03_mva), xycoords='data',
            xytext=(33,0.7), textcoords='data',ha="center",color=colors[0],
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor=colors[0], edgecolor=colors[0])
            )

# Energy 
axes[3].plot(grid_pfr.T[:,0],grid_pfr.get_values('soc_03'),color=colors[1])
axes[3].plot([t_ini,t_end],[0.5]*2,color='#555')
axes[3].plot([t_ini,t_end],[0.0]*2,color=colors[0])
axes[3].plot(grid.T[idx_pert],grid_pfr.get_values('soc_03')[idx_pert],'o',color=colors[5] )
axes[3].plot(grid.T[idx_nadir],grid_pfr.get_values('soc_03')[idx_nadir],'o',color=colors[3])
axes[3].plot(grid.T[idx_nadir_ref],grid_pfr.get_values('soc_03')[idx_nadir_ref],'o',color=colors[2] )
axes[3].plot(grid.T[idx_nadir_ref_end],grid_pfr.get_values('soc_03')[idx_nadir_ref_end],'o',color='#888')

axes[3].annotate("Steady State Energy",
            xy=(35,0.5), xycoords='data',
            xytext=(35,0.3), textcoords='data',ha="center",color='#444',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#444', edgecolor='#444')
            )

axes[3].annotate("Energy Slow Recovery",
            xy=(35,0.03), xycoords='data',
            xytext=(30,0.15), textcoords='data',ha="center",color='#444',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#444', edgecolor='#444')
            )

for ax in axes:
    ax.grid()
    ax.set_xlim(t_ini,t_end)
#axes[1].plot(grid.T,grid.get_values('p_s_03')*S_b_03_mva,label='p_s_03')

axes[0].legend()
axes[1].legend()

axes[0].set_ylabel('Frequency (pu)')
axes[1].set_ylabel('SYN Powers (MW)')
axes[2].set_ylabel('CIG Powers (MW)')
axes[3].set_ylabel('CIG Energy (p.u.)')
axes[3].set_xlabel('Time (s)')

fig.align_ylabels()
fig.tight_layout()

fig.savefig('../'*5 + 'docs/fpfr/figs/freq_powers_energy.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PFR curve and obtained values

In [306]:
fig,axes = plt.subplots(nrows=1)

omega_ini = 0.97
omega_end = 1.05

axes.plot([omega_ini,omega_end],[p_pfr_pu]*2,color='#666')
axes.plot([omega_ref,omega_ref],[-10,10],color='#666')

axes.plot(omegas,p_fs)

axes.plot(grid_pfr.get_values('omega_coi'),grid_pfr.get_values('p_s_03'))
axes.plot(grid_pfr.get_values('omega_coi')[idx_nadir_ref],grid_pfr.get_values('p_s_03')[idx_nadir_ref],'o',color=colors[2])
axes.plot(grid_pfr.get_values('omega_coi')[idx_pert],grid_pfr.get_values('p_s_03')[idx_pert],'o',color=colors[5] )
axes.plot(grid_pfr.get_values('omega_coi')[idx_nadir],grid_pfr.get_values('p_s_03')[idx_nadir],'o',color=colors[3])
axes.plot(grid_pfr.get_values('omega_coi')[idx_nadir_ref_end],grid_pfr.get_values('p_s_03')[idx_nadir_ref_end],'o',color='#888')

#axes.plot(omega_lh,P_f_max,'o')

#axes.plot(omega_ll,0,'o')


#axes.plot([omega_ini,omega_end],[P_f_max]*2,color=colors[0])

axes.annotate("$p_f^{\star}$",
            xy=(0.986,p_pfr_pu), xycoords='data',
            xytext=(0.989,p_pfr_pu*1.2), textcoords='data',ha="center",color='#444',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#666', edgecolor='#666')
            )

axes.annotate("$\omega^{\star}$",
            xy=(omega_ref,-0.044), xycoords='data',
            xytext=(omega_ref-0.005,-0.044), textcoords='data',ha="center",color='#444',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#666', edgecolor='#666')
            )

axes.annotate("$P_f^{\max}$",
            xy=(0.975,P_f_max), xycoords='data',
            xytext=(0.972,0.44), textcoords='data',ha="center",color=colors[0],
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor=colors[0], edgecolor=colors[0])
            )

axes.annotate("Load Switching Transient",
            xy=(1,0.1), xycoords='data',
            xytext=(0.992,0.147), textcoords='data',ha="center",color='#666',
            arrowprops=dict(arrowstyle='-|>, head_width=0.1', facecolor='#666', edgecolor='#666')
            )


#axes.annotate("DB",
#            xy=(1, 0), xycoords='data',
#            xytext=(1-DB/2,-0.01), textcoords='data',ha="center",va="center",
#            arrowprops=dict(arrowstyle='<|-|>, head_width=0.1', facecolor='black',connectionstyle="bar")
#            )

axes.set_xlim(0.97,1.005)
axes.set_ylim(-0.1,P_f_max*1.2)
axes.grid()

axes.set_xlabel('Frequency (pu)')
axes.set_ylabel('Powers (pu)')

fig.align_ylabels()
fig.tight_layout()

fig.savefig('../'*5 + 'docs/fpfr/figs/obtained_droop.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Energy validation

In [307]:
P_cig_mw =grid_pfr.get_values('p_s_03')*S_b_03_mva 
e_pfr_real =np.trapz(P_cig_mw,grid_pfr.T[:,0])

print(f'e_pfr_estimated = {e_pfr_estimated:0.3f} MJ')
print(f'e_pfr_real = {e_pfr_real:0.3f} MJ')

e_pfr_estimated = 2.468 MJ
e_pfr_real = 2.295 MJ


In [286]:
from matplotlib.colors import ListedColormap

plt.close('all')
fig,axes = plt.subplots(nrows=3, figsize=(6.4, 4.8*1.5))

#ax1 = fig.add_axes([0.10,0.10,0.70,0.85])

N = 20
p_pert_min = -0.25
p_pert_max =  0.25

color_list = []
for Dp_pert in np.linspace(p_pert_min,p_pert_max,N):
    
    color = (0,0,0)
    if Dp_pert<0.0:
        color = tuple(np.array([217, 83,  79,255])/255*Dp_pert/(p_pert_min))
    if Dp_pert>=0.0:
        color = tuple(np.array([51, 122, 183,255])/255*Dp_pert/(p_pert_max)) 
    color_list += [color]
    
cmap = ListedColormap(color_list)


Dp_pert_designed = -0.2
t_ini = 0
t_end = 45


axes[0].plot([t_ini,t_end],2*[omega_ref],color='black',lw=lw)
axes[0].plot([t_ini,t_end],2*[omega_h_ref],color='black',lw=lw)
 
axes[1].plot([t_ini,t_end],[p_pfr_pu*S_b_03_mva]*2,color='#666')
axes[1].plot([t_ini,t_end],[P_f_max*S_b_03_mva]*2,color='#444')

axes[1].plot([t_ini,t_end],[-p_pfr_pu*S_b_03_mva]*2,color='#666')
axes[1].plot([t_ini,t_end],[-P_f_max*S_b_03_mva]*2,color='#444')

color_list = []
i = 0
for Dp_pert in np.linspace(p_pert_min,p_pert_max,N):
    
    color = (0,0,0,0)
    if Dp_pert>0.0:
        color = tuple(np.array([217, 83,  79,255])/255*Dp_pert/(p_pert_max))
    if Dp_pert<=0.0:
        color = tuple(np.array([51, 122, 183,255])/255*Dp_pert/(p_pert_min)) 
    color_list += [color]
    
    lw = 1.0
    if abs(Dp_pert-Dp_pert_designed)<0.01:
        lw = 2
        color = colors[1]
    
    params_pfr = params.copy()
    params_pfr.update({'DB_03':DB,'Droop_03':Droop,'P_f_max_03':P_f_max,'P_f_min_03':-P_f_max})
    grid_pfr = onehalf_class()
    gt.change_line(grid_pfr,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
    gt.change_line(grid_pfr,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
    time_0 = time.time()
    grid_pfr.initialize([params_pfr],'xy_0.json',compile=True)
    grid_pfr.run([{'t_end':1.0,'Dt':0.01,'decimation':1}])
    grid_pfr.run([{'t_end':10.0,'P_01':P_load_0_mva*(0-Dp_pert)*1e6}])
    grid_pfr.run([{'t_end':60.1,'Dt':0.1}])
    grid_pfr.post();
    
    # Frequency
    axes[0].plot(grid_pfr.T,grid_pfr.get_values('omega_coi'),label='with PFR',color=color,lw=lw)

    # CIG Power
    axes[1].plot(grid_pfr.T,grid_pfr.get_values('p_s_03')*S_b_03_mva,color=color,lw=lw)

    # Energy 
    axes[2].plot(grid_pfr.T[:,0],grid_pfr.get_values('soc_03'),color=color,lw=lw)

    for ax in axes:
        ax.grid()
        ax.set_xlim(t_ini,t_end)
#axes[1].plot(grid.T,grid.get_values('p_s_03')*S_b_03_mva,label='p_s_03')

#axes[0].legend()
#axes[1].legend()

axes[0].set_ylabel('Frequency (pu)')
axes[1].set_ylabel('CIG Powers (MW)')
axes[2].set_ylabel('CIG Energy (p.u.)')
axes[2].set_xlabel('Time (s)')

fig.align_ylabels()
fig.tight_layout()

cmap = ListedColormap(color_list)
#cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=p_pert_min, vmax=p_pert_max)

#fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
#             cax=axes[1], orientation='horizontal', label='Some Units')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes.ravel().tolist(), 
             label='Load Power Increment (pu)', shrink=0.7)

fig.savefig('../'*5 + 'docs/fpfr/figs/sweep_load.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [209]:
cmap

ValueError: Invalid RGBA argument: 0.8509803921568627

ValueError: Invalid RGBA argument: 0.8509803921568627

In [203]:
import matplotlib as mpl

fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=p_pert_min, vmax=p_pert_max)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=axes[3], orientation='horizontal', label='Some Units')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [183]:
fig, ax = plt.subplots()

data = np.array([0,2,3,4])

cax = ax.imshow(data, cmap=cmap)
ax.set_title('Gaussian noise with horizontal colorbar')

cbar = fig.colorbar(cax, ticks=[-1, 0, 1], orientation='horizontal')
cbar.ax.set_xticklabels(['Low', 'Medium', 'High'])  # horizontal colorbar

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Invalid shape (4,) for image data

In [150]:
color_list

[0.8509803921568627,
 0.3254901960784314,
 0.30980392156862746,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2,
 0.47843137254901963,
 0.7176470588235294,
 1.0]

In [184]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

x = np.linspace(0, 5, 100)
N = 21
cmap = plt.get_cmap('jet',N)

fig = plt.figure(figsize=(8,6))
ax1 = fig.add_axes([0.10,0.10,0.70,0.85])

for i,n in enumerate(np.linspace(0,2,N)):
    y = np.sin(x)*x**n
    ax1.plot(x,y,c=cmap(i))

plt.xlabel('x')
plt.ylabel('y')

norm = mpl.colors.Normalize(vmin=0,vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, ticks=np.linspace(0,2,N), 
             boundaries=np.arange(-0.05,2.1,.1))


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PFR power at point h

In [10]:
t_pert = 1.0
omega_nadir_ref = 0.987
S_base_cig_mva = grid.get_value('S_n_03')/1e6
Droop = 0.02 # in CIG base

omega_hd = omega_nadir_ref

idx_pert = np.searchsorted(grid.T[:,0],t_pert)
idx_nadir = np.argmin(grid.get_values('omega_coi'))
t_nadir = grid.T[idx_nadir]
p_m_pert = grid.get_values('p_m_01')[idx_pert]*S_b_01_mva
omega_nadir = grid.get_values('omega_coi')[idx_nadir]

t_nadir_ref = grid.T[idx_pert:idx_nadir,0][np.argmin(grid.get_values('omega_coi')[idx_pert:idx_nadir]>omega_nadir_ref)]
idx_nadir_ref = np.searchsorted(grid.T[:,0],t_nadir_ref)

P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
p_pfr_mw = P_load[idx_nadir_ref] - grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva

P_f_max = p_pfr_mw/S_base_cig_mva  # pu in CIG power base
Domega = -Droop*P_f_max 
omega_ld = omega_hd - Domega

DB = (1 - omega_ld)*2

print(f'omega_ld = {omega_ld:0.4f}, omega_hd = {omega_hd:0.4f}')
print(f'P_f_max = {P_f_max:0.4f}, DB = {DB:0.4f}')

omegas = np.array([      0,omega_hd, omega_ld,1])
p_fs   = np.array([P_f_max, P_f_max,        0,0])

params_pfr = params.copy()
params_pfr.update({'DB_03':DB,'Droop_03':Droop,'P_f_max_03':P_f_max})
grid_pfr = onehalf_class()
gt.change_line(grid_pfr,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid_pfr,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid_pfr.initialize([params_pfr],'xy_0.json',compile=True)
grid_pfr.run([{'t_end':1.0,'Dt':0.01,'decimation':1}])
grid_pfr.run([{'t_end':t_nadir_ref,'P_02':-9e6}])
grid_pfr.run([{'t_end':10.0,'P_02':-9e6}])
grid_pfr.run([{'t_end':30.0,'Dt':1}])
grid_pfr.post();


fig,axes = plt.subplots(nrows=2)
axes[0].plot(grid.T,grid.get_values('omega_coi'),label='omega_coi')
axes[0].plot(grid_pfr.T,grid_pfr.get_values('omega_coi'),label='omega_coi')

axes[0].plot(grid.T[idx_pert],grid.get_values('omega_coi')[idx_pert],'o' )
axes[0].plot(grid.T[idx_nadir],grid.get_values('omega_coi')[idx_nadir],'o' )
axes[0].plot(grid.T[idx_nadir_ref],grid.get_values('omega_coi')[idx_nadir_ref],'o' )

P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
axes[1].plot(grid.T,P_load,label='p_load')

axes[1].plot(grid.T,grid.get_values('p_m_01')*S_b_01_mva,label='p_m_01')
axes[1].plot(grid_pfr.T,grid_pfr.get_values('p_m_01')*S_b_01_mva,label='p_m_01')

axes[1].plot(grid.T[idx_nadir_ref],grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva,'o')

#axes[1].plot(grid.T,grid.get_values('p_s_03')*S_b_03_mva,label='p_s_03')
axes[1].legend()



omega_ld = 0.9909, omega_hd = 0.9870
P_f_max = 0.1934, DB = 0.0183


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig,axes = plt.subplots(nrows=1)
axes.plot(omegas,p_fs)

axes.plot(grid_pfr.get_values('omega_coi'),grid_pfr.get_values('p_s_03'))
axes.set_xlim(0.98,1.01)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.98, 1.01)

In [12]:
P_load[idx_nadir_ref]

9.003910944225773

In [13]:
## Without PFR by CIG
Dp_pert = -0.2 # p.u. installed power capacity base 
t_pert = 1.0

grid = onehalf_class()
gt.change_line(grid,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid.initialize([params],'xy_0.json',compile=True)
P_load_0_mva = grid.get_value('P_02')/1e6
grid.run([{'t_end':t_pert,'Dt':0.01,'decimation':1}])
grid.run([{'t_end':10.0,'P_01':P_load_0_mva*(0-Dp_pert)*1e6}])
grid.run([{'t_end':60.1,'Dt':0.1}])
grid.post();
 
## Design of PFR by CIG
omega_nadir_ref = 0.982
Droop = 0.01 # in CIG base
mid_pos = 0.1
S_n_syn_mva = grid.get_value('S_n_01')/1e6
S_n_cig_mva = grid.get_value('S_n_03')/1e6
S_n_sys_mva = S_n_syn_mva + S_n_cig_mva


idx_pert = np.searchsorted(grid.T[:,0],t_pert)
idx_nadir = np.argmin(grid.get_values('omega_coi'))
t_nadir = grid.T[idx_nadir]
p_m_pert = grid.get_values('p_m_01')[idx_pert]*S_b_01_mva
omega_nadir = grid.get_values('omega_coi')[idx_nadir]

t_nadir_ref = grid.T[idx_pert:idx_nadir,0][np.argmin(grid.get_values('omega_coi')[idx_pert:idx_nadir]>omega_nadir_ref)]
idx_nadir_ref = np.searchsorted(grid.T[:,0],t_nadir_ref)

t_nadir_ref_end = grid.T[idx_nadir:,0][np.argmin(grid.get_values('omega_coi')[idx_nadir:]<omega_nadir_ref)]
idx_nadir_ref_end = np.searchsorted(grid.T[:,0],t_nadir_ref_end)

P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
p_pfr_mw = P_load[idx_nadir_ref] - grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva

omega_md = omega_nadir_ref
P_f_max = p_pfr_mw/S_n_cig_mva  # pu in CIG power base

omega_hd = omega_md - Droop*P_f_max*(1-mid_pos)
omega_ld = omega_md + Droop*P_f_max*mid_pos

DB = (1 - omega_ld)*2

print(f'omega_ld = {omega_ld:0.4f}, omega_md = {omega_md:0.4f}, omega_hd = {omega_hd:0.4f}')
print(f'P_f_max = {P_f_max:0.4f}, DB = {DB:0.4f}')

## Performance test
Dp_pert = -0.2

params_pfr = params.copy()
params_pfr.update({'DB_03':DB,'Droop_03':Droop,'P_f_max_03':P_f_max})
grid_pfr = onehalf_class()
gt.change_line(grid_pfr,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid_pfr,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid_pfr.initialize([params_pfr],'xy_0.json',compile=True)
grid_pfr.run([{'t_end':1.0,'Dt':0.01,'decimation':1}])
grid_pfr.run([{'t_end':10.0,'P_01':P_load_0_mva*(0-Dp_pert)*1e6}])
grid_pfr.run([{'t_end':60.1,'Dt':0.1}])
grid_pfr.post();

fig,axes = plt.subplots(nrows=3, figsize=(6.4, 4.8*1.5))
axes[0].plot(grid.T,grid.get_values('omega_coi'),label='w/o PFR',color=colors[1])
axes[0].plot(grid_pfr.T,grid_pfr.get_values('omega_coi'),label='with PFR',color=colors[2])

axes[0].plot(grid.T[idx_pert],grid.get_values('omega_coi')[idx_pert],'o',color=colors[5] )
axes[0].plot(grid.T[idx_nadir],grid.get_values('omega_coi')[idx_nadir],'o',color=colors[3])
axes[0].plot(grid.T[idx_nadir_ref],grid.get_values('omega_coi')[idx_nadir_ref],'o',color=colors[2] )


P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
axes[1].plot(grid.T,P_load,label='$p_l$')

axes[1].plot(grid.T,grid.get_values('p_m_01')*S_b_01_mva,label='$p_m$ w/o PFR')
axes[1].plot(grid_pfr.T,grid_pfr.get_values('p_m_01')*S_b_01_mva,label='$p_m$ with PFR')


axes[1].plot(grid.T[idx_pert],grid.get_values('p_m_01')[idx_pert]*S_b_01_mva,'o',color=colors[5] )
axes[1].plot(grid.T[idx_nadir],grid.get_values('p_m_01')[idx_nadir]*S_b_01_mva,'o',color=colors[3])
axes[1].plot(grid.T[idx_nadir_ref],grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva,'o',color=colors[2] )


axes[2].plot(grid_pfr.T,grid_pfr.get_values('p_s_03')*S_b_03_mva)
axes[2].plot(grid.T[idx_pert],grid_pfr.get_values('p_s_03')[idx_pert]*S_b_03_mva,'o',color=colors[5] )
axes[2].plot(grid.T[idx_nadir],grid_pfr.get_values('p_s_03')[idx_nadir]*S_b_03_mva,'o',color=colors[3])
axes[2].plot(grid.T[idx_nadir_ref],grid_pfr.get_values('p_s_03')[idx_nadir_ref]*S_b_03_mva,'o',color=colors[2] )
axes[2].plot(grid.T[idx_nadir_ref_end],grid_pfr.get_values('p_s_03')[idx_nadir_ref_end]*S_b_03_mva,'o',color='#888')
axes[2].plot(grid.T,grid.T*0+P_f_max*S_b_03_mva,color='#888')


for ax in axes:
    ax.grid()
    ax.set_xlim(0,45)
#axes[1].plot(grid.T,grid.get_values('p_s_03')*S_b_03_mva,label='p_s_03')

axes[0].legend()
axes[1].legend()

axes[0].set_ylabel('Frequency (pu)')
axes[1].set_ylabel('SYN Powers (MW)')
axes[2].set_ylabel('CIG Powers (MW)')
axes[2].set_xlabel('Time (s)')

fig.align_ylabels()
fig.tight_layout()


omegas = np.array([      0,omega_hd, omega_ld,1])
p_fs   = np.array([P_f_max, P_f_max,        0,0])

omega_ld = 0.9824, omega_md = 0.9820, omega_hd = 0.9787
P_f_max = 0.3700, DB = 0.0353


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
t_pert = 1.0
idx_pert = np.searchsorted(grid.T[:,0],t_pert)
idx_nadir = np.argmin(grid.get_values('omega_coi'))
t_nadir = grid.T[idx_nadir]
p_m_pert = grid.get_values('p_m_01')[idx_pert]*S_b_01_mva
omega_nadir = grid.get_values('omega_coi')[idx_nadir]
omega_nadir_ref = 0.987
t_nadir_ref = grid.T[idx_pert:idx_nadir,0][np.argmin(grid.get_values('omega_coi')[idx_pert:idx_nadir]>omega_nadir_ref)]
idx_nadir_ref = np.searchsorted(grid.T[:,0],t_nadir_ref)

P_load = grid.get_values('p_g_01')*S_b_01_mva + grid.get_values('p_s_03')*S_b_03_mva 
p_pfr = P_load[idx_nadir_ref] - grid.get_values('p_m_01')[idx_nadir_ref]*S_b_01_mva
p_pfr

Domega = 0.0005
omega_DB_min = omega_nadir_ref + Domega
omega_DB_mid = omega_nadir_ref + Domega*0.5
omega_p_f_max = omega_nadir_ref - Domega
DB = 2*(1 - omega_DB_min)
# Droop = freq_pu/p_pu
Droop = (omega_DB_min-omega_DB_mid)/(p_pfr/S_b_03_mva)
P_f_max =1/Droop*(omega_DB_min - omega_p_f_max)

params_pfr = params.copy()
params_pfr.update({'DB_03':DB,'Droop_03':Droop,'P_f_max_03':P_f_max})
grid_pfr = onehalf_class()
gt.change_line(grid_pfr,'01','02',X_pu=0.04,R_pu=0.001,S_mva=20)
gt.change_line(grid_pfr,'02','03',X_pu=0.04,R_pu=0.001,S_mva=20)
time_0 = time.time()
grid_pfr.initialize([params_pfr],'xy_0.json',compile=True)
grid_pfr.run([{'t_end':1.0,'Dt':0.01,'decimation':1}])
grid_pfr.run([{'t_end':t_nadir_ref,'P_02':-9e6}])
grid_pfr.run([{'t_end':10.0,'P_02':-9e6+0*p_pfr*1e6}])
grid_pfr.run([{'t_end':60.0,'Dt':1}])
grid_pfr.post();
print(f'p_pfr = {p_pfr}')

p_pfr = 1.3394013652550338


In [397]:
DB

0.025000000000000133

In [590]:
P_f_max

1.163715642842888

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [594]:
fig,axes = plt.subplots()

axes.plot(grid_pfr.T,grid_pfr.get_values('v_u_03'),label='v_u_03')

axes.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [535]:
omega_pu = np.array([0,])


350

In [514]:
fig,axes = plt.subplots()
axes.plot(grid.T,grid.get_values('V_01'),label='V_01')
axes.plot(grid.T,grid.get_values('V_02'),label='V_02')
axes.plot(grid.T,grid.get_values('V_03'),label='V_03')

axes.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [495]:
fig,axes = plt.subplots()
axes.plot(grid.T,grid.get_values('v_u_03'),label='v_u_03')
axes.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [316]:
#!/usr/bin/python3
import requests
import json
import pprint

## Request and header parameters
token = 'gYw1vzE50Vc41NKzNf9NuUlH6Em3G5eFwoofcAynyk/axgyy++IgUwVODd2O3PBMVx7CNoSRDGEcDSdVAo1mCLQ3DzkJLBIzj1fGpk1bSGHHpexlIyQ+yB8jzO+0j+/N'
url = 'https://api.gridradar.net/query'

headers = {
    'Content-type': 'application/json', 
    'Authorization': 'Bearer '+token
}

## API query parameters
request = {
  "metric":"frequency-ucte-median-1s",
#  "pmus":"neustadt,dresden", 
#  "from":"2021-05-26T02:36:17.3Z", 
#  "to":"2021-05-26T02:36:17.6Z", 
  "aggr":"1s", 
  "format":"json",
  "ts":"rfc3339"
}

## Converting the Python dictionary to a JSON string
json_request = json.dumps(request)

## Request execution and response reception
response = requests.post(url, data=json_request, headers=headers)

## Converting the JSON response string to a Python dictionary
result_dict = json.loads(response.content)

## Pretty print response dictionary
pprint.pprint(result_dict)

{'columns': ['code', 'msg'],
 'data': [[400,
           'Query contains not permitted arguments for account type Free: '
           'from']]}
